# McStasRead demo

In [ ]:
from read_example import make_instrument, plot

In [ ]:
instr = make_instrument()
instr.show_parameters()

In [ ]:
instr.set_parameters(wavelength=1.8, delta_wavelength=1.3)
instr.settings(ncount=1E6)

In [ ]:
instr.show_diagram()

### Run simulation

In [ ]:
data = instr.backengine()

In [ ]:
data

### Histogram data to display

In [ ]:
hist_data = [mon.make_2d(mon.variables[1], "y") for mon in data]

In [ ]:
import mcstasscript as ms
ms.make_sub_plot(hist_data)

In [ ]:
file_path = data[0].original_data_location
print(file_path)

In [ ]:
%matplotlib widget

### Show list of components found in NeXus file

In [ ]:
import mcstastox

with mcstastox.Read(file_path) as loaded_data:
    loaded_data.show_components()

#### Show list of these with available data

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    loaded_data.show_components_with_data()

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    variables = loaded_data.get_component_variables("Square_1")
    print(variables)

#### Show list of these with geometry information

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    loaded_data.show_components_with_geometry()

#### Show monitors with pixel ID's

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    loaded_data.show_components_with_ids()

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    print(loaded_data.get_components_with_ids())

### Export to Scipp object
Using the *export_scipp* method we get a scipp DataGroup that contains:
- events : the data
- positions : positions of the pixel ids
- bank_ids : pixel id range for each detector bank
- bank_names : names of the loaded detector banks

This structure requires a little more knowledge to work with than the simple export, but saves space and provides more flexibility.

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    scipp_object = loaded_data.export_scipp(source_name="source",
                                            sample_name="sample_position")

In [ ]:
scipp_object

In [ ]:
scipp_object["bank_names"]

In [ ]:
scipp_object["bank_ids"]

In [ ]:
scipp_object["positions"]

In [ ]:
scipp_object["events"]

## Plot pixels with intensities
With this setup we can plot the total intensity in each pixel rather than all events individually.

In [ ]:
import plopp as pp

pp.scatter3d(scipp_object["events"].hist(), pos='position', size=0.015, cbar=True, norm="log")

### Perform coordinate transforms
Coordinate transformations can be done almost as before, they just need to be summed over all *pixel_id* when plotted.

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic

event_object = scipp_object["events"]

# McStas provides absolute time, not time of flight
event_object.bins.coords["tof"] = event_object.bins.coords["t"]

graph = {**beamline(scatter=True), **elastic("tof")}

In [ ]:
event_object = event_object.transform_coords("dspacing", graph=graph)

Note extra .sum("pixel_id") in plot.

In [ ]:
event_object.hist(dspacing=150).sum("pixel_id").plot(norm="log")